In [18]:
import stats_funcs.likelihood_ratio as lr

import json
import pandas as pd
import numpy as np

from scipy.integrate import quad
from scipy.optimize import minimize

# Current solution

In [19]:
sample = json.load(open('data/snls_snia.json'))

df = pd.DataFrame(sample)

In [20]:
x0 = [0.76, 0.24, -1, 71] #['Omega_l', 'Omega_m', 'w',  'Hubble']

minimize(lr.h, x0 = x0, args = ({}, df), method = 'Nelder-Mead', tol = 1e-6, bounds = ((0,1), (0,1), (-1.5, 0), (0, None)), options = {'maxiter': 10000})

 final_simplex: (array([[ 0.5616612 ,  0.        , -1.10103514, 70.15611476],
       [ 0.56166111,  0.        , -1.10103542, 70.15611557],
       [ 0.56166108,  0.        , -1.10103544, 70.15611549],
       [ 0.56166127,  0.        , -1.10103503, 70.15611506],
       [ 0.56166114,  0.        , -1.10103539, 70.1561155 ]]), array([110.94386128, 110.94386128, 110.94386128, 110.94386128,
       110.94386128]))
           fun: 110.94386128057224
       message: 'Optimization terminated successfully.'
          nfev: 638
           nit: 378
        status: 0
       success: True
             x: array([ 0.5616612 ,  0.        , -1.10103514, 70.15611476])

# With array input

In [21]:
def Dc(z: float, params) -> float:
    """
    Calculates the adimensional comoving distance using the input parameters.

    Parameters:
        z (float): The redshift value.
        Omega_l (float): The value of the cosmological constant.
        Omega_m (float): The value of the matter density parameter.
        w (float): The value of the dark energy equation of state.

    Returns:
        The adimensional comoving distance.
        
    Notes:
        The adimensional comoving distance is the distance that would be traveled 
        by a photon emitted at redshift z, if the universe were not expanding, 
        divided by the Hubble distance. This function calculates the comoving 
        distance by performing a numerical integration of the function f(z, Omega_l, 
        Omega_m, w) over the redshift range [0, z], where f(z, Omega_l, Omega_m, w) 
        is defined as:

        f(z, Omega_l, Omega_m, w) = 1 / sqrt(Omega_l*(1 + z)**(3 + 3*w) + 
                                              Omega_m*(1 + z)**3 + 
                                              Omega_k*(1 + z)**2 + 
                                              9.2e-5*(1 + z)**4)

        where Omega_k = 1 - (Omega_l + Omega_m + 9.2e-5) represents the curvature of 
        the universe. The result is returned as an adimensional quantity, which 
        can be converted to physical units of length by multiplying by the Hubble 
        distance.
    """
    
    Omega_k = 1 - (params[0] + params[1] + 9.2e-5)

    def f(z, a, b, c):
        return 1/(np.sqrt(a*(1 + z)**(3 + 3*c) + b*(1 + z)**3 + Omega_k*(1 + z)**2 + 9.2e-5*(1 + z)**4))
    
    result = quad(f, 0, z, args = (params[0], params[1], params[2]))[0]

    return result

def Dt(z: float, params) -> float:
    """
    Calculates the temporal comoving distance using the input parameters.

    Parameters:
        z (float): The redshift value.
        Omega_l (float): The value of the cosmological constant.
        Omega_m (float): The value of the matter density parameter.
        w (float): The value of the dark energy equation of state.

    Returns:
        The temporal comoving distance.
        
    Notes:  
        -If Omega_k is greater than 0, the universe is positively curved like a 
        sphere, and the correction factor involves a sine function.

        -If Omega_k is equal to 0, the universe is flat, and no correction factor 
        is necessary.

        -If Omega_k is less than 0, the universe is negatively curved like a saddle, 
        and the correction factor involves a hyperbolic sine function.
    """
    
    Omega_k = 1 - (params[0] + params[1] + 9.2e-5) # Omega_l + Omega_m + Omega_r + Omega_k = 1
    
    if Omega_k > 0:
        return np.sin(np.sqrt(Omega_k)*Dc(z, params))/np.sqrt(Omega_k)
    elif Omega_k == 0:
        return Dc(z, params)
    elif Omega_k:
        return np.sinh(np.sqrt(abs(Omega_k))*Dc(z, params))/np.sqrt(abs(Omega_k))

def Dl(z: float, params) -> float:
    """
    Calculates the luminosity distance using the input parameters.

    Parameters:
        z (float): The redshift value.
        Omega_l (float): The value of the cosmological constant.
        Omega_m (float): The value of the matter density parameter.
        w (float): The value of the dark energy equation of state.

    Returns:
        The luminosity distance.
    """
    
    return (1 + z) * Dt(z, params)
               
def mu(z: float, params) -> float:
    """
    Calculates the distance modulus using the input parameters.

    Parameters:
        z (float): The redshift value.
        Omega_l (float): The value of the cosmological constant.
        Omega_m (float): The value of the matter density parameter.
        w (float): The value of the dark energy equation of state.
        Hubble (float): The Hubble parameter.

    Returns:
        The distance modulus.
    """
    
    c = 3e5 #(km/s)
    return 5*np.log10(Dl(z, params)) + 25 + 5*np.log10(c/ params[3])

def chi2_i(mu_o: float, sigma_o: float, z_obs: float, params) -> float:
    """
    Calculates the chi-squared value for a single data point using the input parameters.

    Parameters:
        mu_o (float): The observed distance modulus.
        sigma_o (float): The uncertainty in the observed distance modulus.
        z_obs (float): The observed redshift value.
        Omega_l (float): The value of the cosmological constant.
        Omega_m (float): The value of the matter density parameter.
        w (float): The value of the dark energy equation of state.
        Hubble (float): The Hubble parameter.

    Returns:
        The chi-squared value for a single data point.
    """
    
    _mu = mu(z_obs, params)
    return ((_mu - mu_o)**2)/sigma_o**2

def chi2(params, df): 
    """
    Calculates the chi-squared value for a set of data points using the input parameters.

    Parameters:
        Omega_l (float): The value of the cosmological constant.
        Omega_m (float): The value of the matter density parameter.
        w (float): The value of the dark energy equation of state.
        Hubble (float): The Hubble parameter.
        df (pandas.DataFrame): The dataframe containing the observed redshift values, distance moduli, and uncertainties.

    Returns:
        The chi-squared value for the set of data points.
    """
    values = []
    for idx in df.index:
        values.append(chi2_i(df.mu[idx], df.sigma[idx], df.z[idx], params))
    result = sum(values)
    return result

In [22]:
minimize(chi2, x0 = x0, args = (df), method = 'Nelder-Mead', tol = 1e-6, bounds = ((0,1), (0,1), (-1.5, 0), (0, None)), options = {'maxiter': 10000})

 final_simplex: (array([[ 0.5616612 ,  0.        , -1.10103514, 70.15611476],
       [ 0.56166111,  0.        , -1.10103542, 70.15611557],
       [ 0.56166108,  0.        , -1.10103544, 70.15611549],
       [ 0.56166127,  0.        , -1.10103503, 70.15611506],
       [ 0.56166114,  0.        , -1.10103539, 70.1561155 ]]), array([110.94386128, 110.94386128, 110.94386128, 110.94386128,
       110.94386128]))
           fun: 110.94386128057224
       message: 'Optimization terminated successfully.'
          nfev: 638
           nit: 378
        status: 0
       success: True
             x: array([ 0.5616612 ,  0.        , -1.10103514, 70.15611476])

As we can see, the output received from the redefined functions without the wrapper converged to the same values. This means that the error must be inside the cosmological functions.